# A* algoritam

## Korišćenje A* za pronalazak puta u grafu

In [1]:
adjacency_list = {
    'A': [('B', 10), ('C', 5), ('D', 10)],
    'B': [('D', 1)],
    'C': [('D', 2)]
}

In [41]:
from collections import deque

class Graph:
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list
        
    def __str__(self):
        return str(self.adjacency_list)
        
    def get_neighbors(self, v):
        return self.adjacency_list[v]
    
    # Heuristika
    def h(self, n):
        H = {
                'A' : 1,
                'B' : 1,
                'C' : 1,
                'D': 1
                }

        return H[n]

    # Pronalazenje najkraceg puta pomocu algoritma A*
    def astar(self, start, stop):

        # Zatvorena lista je inicijalno prazna, a otvorena lista sadrzi samo polazni cvor
        open_list = set([start])
        closed_list = set([])

        # g sadrzi tekuce udaljenosti od polaznog cvora (start) do ostalih cvorova, ukoliko se cvor ne nalazi
        # u mapi, podrazumevana vrednost udaljenosti je beskonacno
        g = {}
        
        # Udaljenost polaznog cvora od samog sebe je 0
        g[start] = 0

        # Mapa parents cuva roditelje cvorova
        parents = {}
        parents[start] = start

        # Izvrsavaj dok god ima elemenata u otvorenoj listi
        while len(open_list) > 0:

            n = None

            for v in open_list:
                if n == None or g[v] + self.h(v) < g[n] +  self.h(n):
                    n = v;

            if n == None:
                print('Trazeni put ne postoji')
                return None

            # Ako je n ciljni cvor, izvesti o uspehu i vrati resenje konstruisuci put
            # od polaznog do ciljnog cvora (iduci unazad — od ciljnog cvora).
            if n == stop:
                path = []

                # do-while petlja ne postoji u Python-u
                while parents[n] != n:
                    path.append(n)
                    n = parents[n]

                # Dodajemo polazni cvor
                path.append(start)

                path.reverse()
                
                print('Pronadjen je put: {}'.format(path))
                return path

            # Za svaki cvor m koji je direktno dostupan izn𝑛 uradi sledece:
            for (m, weight) in self.get_neighbors(n):

                # Ako m nije ni u otvorenoj ni u zatvorenoj listi, dodaj ga u otvorenu listu
                # i oznaci n kao njegovog roditelja.
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight

                # Inace, proveri da li je put od polaznog cvora do cvora m preko
                # cvora n bolji (kraci ili jeftiniji) od postojeceg puta do m
                # (trenutna vrednost g(m)). Ako jeste, promeni informaciju o roditelju
                # cvora m na cvor n i azuriraj vrednosti g(m), a ako je
                # m bio u zatvorenoj listi, prebaci ga u otvorenu.
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n

                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)

            # Izbaci n iz otvorene liste i dodaj ga u zatvorenu listu
            open_list.remove(n)
            closed_list.add(n)

        #  Obavesti da trazeni put ne postoji (otvorena lista je prazna i uspeh nije prijavljen).
        print('Trazeni put ne postoji')
        return None

In [42]:
g = Graph(adjacency_list)
g.astar('A','D')

Pronadjen je put: ['A', 'C', 'D']


['A', 'C', 'D']

## Korišćenje A* za rešavanje Lojdove slagalice

In [9]:
from collections import deque
import copy

class Game:
    def __init__(self, start_state):
        self.start_state = self.serialize(start_state)
        self.end_state = self.serialize([[1,2,3],[4,5,6],[7,8,0]])
    
    def __str__(self):
        return str(self.adjacency_list)
    
    def serialize(self, state):
        serialized = []
        for i in range(len(state)):
            for j in range(len(state[i])):
                serialized.append(str(state[i][j]))
        return ':'.join(serialized)
    
    def deserialize(self, serialized):
        serialized_list = serialized.split(':')
        deserialized = [
            serialized_list[:3],
            serialized_list[3:6],
            serialized_list[6:]
        ]
        
        deserialized = [[int(x) for x in row] for row in deserialized]
        
        return deserialized
    
    def get_neighbors(self, state):
        deserialized = self.deserialize(state)
        
        neighbors = []
        
        blank_i = -1
        blank_j = -1
        
        for i in range(len(deserialized)):
            for j in range(len(deserialized[i])):
                if deserialized[i][j] == 0:
                    blank_i = i
                    blank_j = j
                    break
                    
        i = blank_i
        j = blank_j
                    
        if i > 0:
                new_matrix = copy.deepcopy(deserialized)
                new_matrix[i][j] = new_matrix[i - 1][j]
                new_matrix[i - 1][j] = 0
                neighbors.append(self.serialize(new_matrix))
                
        if i < 2:
                new_matrix = copy.deepcopy(deserialized)
                new_matrix[i][j] = new_matrix[i + 1][j]
                new_matrix[i + 1][j] = 0
                neighbors.append(self.serialize(new_matrix))
            
        if j > 0:
            new_matrix = copy.deepcopy(deserialized)
            new_matrix[i][j] = new_matrix[i][j - 1]
            new_matrix[i][j - 1] = 0
            neighbors.append(self.serialize(new_matrix))
            
        if j < 2:
                new_matrix = copy.deepcopy(deserialized)
                new_matrix[i][j] = new_matrix[i][j + 1]
                new_matrix[i][j + 1] = 0
                neighbors.append(self.serialize(new_matrix))
                
        return zip(neighbors, [1 for x in neighbors])
    
    # Odredjivanje heuristike za state
    # Koristi se Menhetn rastojanje
    # Heuristika se odredjuje kao zbir pomeraja koji su potrebni za svako polje da bi se naslo na svom mestu
    # (kod Menhetn rastojanja zanemaruje se situacija da nije moguce preko nekih polja preci, odnosno, u ovom primeru da
    # druga polja mogu biti popunjena i pomeraje nije moguce napraviti)
    # Na primer, broj 5 treba da bude na poziciji (1, 1) u matrici
    # Ukoliko vazi da je mat[0][2] = 5, onda je potrebno napraviti (minimum) 2 pomeraja da bi se 5 nasao na (1, 1),
    # odnosno abs(0 - 1) + abs(2 - 1)
    def h(self, state):
        deserialized = self.deserialize(state)
        
        H = 0
        for i in range(0, 3):
            for j in range(0, 3):
                if deserialized[i][j] == 0:
                    H += 2 - i + 2 - j
                elif deserialized[i][j] % 3 == 0:
                    H += 2 - j + abs(deserialized[i][j] / 3 - 1 - i)
                else:
                    H += abs(deserialized[i][j] / 3 - i) + abs(deserialized[i][j] % 3 - j - 1)
                    
        return H
    
    def astar(self):
        open_list = set([self.start_state])
        closed_list = set([])
        
        D = {}
        D[self.start_state] = 0
        
        parent = {}
        parent[self.start_state] = None
        
        iterations_number = 0
        
        while len(open_list) > 0:
            n = None
            min_d = float('inf')
            
            for v in open_list:
                if D[v] + self.h(v) < min_d:
                    n = v
                    min_d = D[v] + self.h(v)
                    
            if n == None:
                print('Nema puta')
                return None
            
            if n == self.end_state:
                path = []
                
                while n != None:
                    path.append(n)
                    n = parent[n]
                    
                path.reverse()
                
                print('Put nadjen u {} iteracija'.format(iterations_number))
                return [self.deserialize(x) for x in path]
            
            for (m, weight) in self.get_neighbors(n):
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parent[m] = n
                    D[m] = D[n] + weight
                else:
                    if D[m] > D[n] + weight:
                        D[m] = D[n] + weight
                        parent[m] = n
                        
                        if m in closed_list:
                            open_list.add(m)
                            closed_list.remove(m)
            
            closed_list.add(n)
            open_list.remove(n)
            
            iterations_number += 1
            
        print('Nema puta')
        return None

In [10]:
T = Game([
    [2,3,5],
    [1,4,0],
    [7,8,6]
])

In [11]:
T.astar() # 13 iteracija

Put nadjen u 13 iteracija


[[[2, 3, 5], [1, 4, 0], [7, 8, 6]],
 [[2, 3, 0], [1, 4, 5], [7, 8, 6]],
 [[2, 0, 3], [1, 4, 5], [7, 8, 6]],
 [[0, 2, 3], [1, 4, 5], [7, 8, 6]],
 [[1, 2, 3], [0, 4, 5], [7, 8, 6]],
 [[1, 2, 3], [4, 0, 5], [7, 8, 6]],
 [[1, 2, 3], [4, 5, 0], [7, 8, 6]],
 [[1, 2, 3], [4, 5, 6], [7, 8, 0]]]

## Korišćenje A* za pronalazak puta u lavirintu

In [6]:
import copy

# Prolazna polja: .
# Neprolazna polja: X
# Tesko prolazna polja: # (Tezina 2)
# Start: S
# Cilj: F

class Maze:
    def __init__(self, maze_matrix):
        (self.start, self.finish, self.adjacency_list) = self.convert_to_graph(maze_matrix)
        self.maze_matrix = maze_matrix
        
    def __str__(self):
        return str(self.adjacency_list)
    
    def get_neighbors(self, v):
        return self.adjacency_list[v]
    
    # Pretvaranje matrice lavirinta u graf (listu susedstva)
    def convert_to_graph(self, maze_matrix):
        adjacency_list = {}
        start = None
        finish = None
        
        n = len(maze_matrix)
        m = len(maze_matrix[0])
        
        for i in range(n):
            for j in range(m):
                v = (i, j)
                neighbors = []
                
                if maze_matrix[i][j] != 'X':
                    
                    if maze_matrix[i][j] == 'S':
                        start = v
                    
                    if maze_matrix[i][j] == 'F':
                        finish = v
                    
                    if i > 0:
                        if maze_matrix[i - 1][j] != 'X':
                            w = (i-1, j)
                            if maze_matrix[i - 1][j] == '#':
                                weight = 2
                            else:
                                weight = 1
                            
                            neighbors.append((w, weight))
                            
                    if i < n - 1:
                        if maze_matrix[i + 1][j] != 'X':
                            w = (i+1, j)
                            if maze_matrix[i + 1][j] == '#':
                                weight = 2
                            else:
                                weight = 1
                                
                            neighbors.append((w, weight))
                            
                    if j > 0:
                        if maze_matrix[i][j - 1] != 'X':
                            w = (i, j-1)
                            if maze_matrix[i][j - 1] == '#':
                                weight = 2
                            else:
                                weight = 1
                            neighbors.append((w, weight))
                            
                    if j < m - 1:
                        if maze_matrix[i][j + 1] != 'X':
                            w = (i, j+1)
                            if maze_matrix[i][j + 1] == '#':
                                weight = 2
                            else:
                                weight = 1
                            neighbors.append((w, weight))
                        
                adjacency_list[v] = neighbors
                
        return start, finish, adjacency_list
    
    # Heuristika, Menhetn rastojanje
    # - Testirati i sa euklidskim rastojanjem
    def h(self, v_coords):
        v_x1 = int(v_coords[0])
        v_x2 = int(v_coords[1])
        
        finish_x1 = int(self.finish[0])
        finish_x2 = int(self.finish[1])
        
        return abs(v_x1 - finish_x1) + abs(v_x2 - finish_x2)
    
    # Pronalazenje najkraceg puta pomocu algoritma A*
    def astar(self, start, stop):

        # Zatvorena lista je inicijalno prazna, a otvorena lista sadrzi samo polazni cvor
        open_list = set([start])
        closed_list = set([])

        # g sadrzi tekuce udaljenosti od polaznog cvora (start) do ostalih cvorova, ukoliko se cvor ne nalazi
        # u mapi, podrazumevana vrednost udaljenosti je beskonacno
        g = {}
        
        # Udaljenost polaznog cvora od samog sebe je 0
        g[start] = 0

        # Mapa parents cuva roditelje cvorova
        parents = {}
        parents[start] = start

        # Izvrsavaj dok god ima elemenata u otvorenoj listi
        while len(open_list) > 0:

            n = None

            for v in open_list:
                if n == None or g[v] + self.h(v) < g[n] +  self.h(n):
                    n = v;

            if n == None:
                print('Trazeni put ne postoji')
                return None

            # Ako je n ciljni cvor, izvesti o uspehu i vrati resenje konstruisuci put
            # od polaznog do ciljnog cvora (iduci unazad — od ciljnog cvora).
            if n == stop:
                path = []

                # do-while petlja ne postoji u Python-u
                while parents[n] != n:
                    path.append(n)
                    n = parents[n]

                # Dodajemo polazni cvor
                path.append(start)

                path.reverse()
                
                #print('Pronadjen je put: {}'.format(path))
                return path

            # Za svaki cvor m koji je direktno dostupan izn𝑛 uradi sledece:
            for (m, weight) in self.get_neighbors(n):

                # Ako m nije ni u otvorenoj ni u zatvorenoj listi, dodaj ga u otvorenu listu
                # i oznaci n kao njegovog roditelja.
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight

                # Inace, proveri da li je put od polaznog cvora do cvora m preko
                # cvora n bolji (kraci ili jeftiniji) od postojeceg puta do m
                # (trenutna vrednost g(m)). Ako jeste, promeni informaciju o roditelju
                # cvora m na cvor n i azuriraj vrednosti g(m), a ako je
                # m bio u zatvorenoj listi, prebaci ga u otvorenu.
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n

                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)

            # Izbaci n iz otvorene liste i dodaj ga u zatvorenu listu
            open_list.remove(n)
            closed_list.add(n)

        #  Obavesti da trazeni put ne postoji (otvorena lista je prazna i uspeh nije prijavljen).
        print('Trazeni put ne postoji')
        return None
    
    # Prevodjenje pronadjenog puta u resenje polaznog problema
    # pronalazenja puta u lavirintu
    def solve(self):
        graph_path = self.astar(self.start, self.finish)
        if not graph_path:
            print('No solution!')
            return self.maze_matrix
        else:
            print('Found the path!')
            solved_matrix = copy.deepcopy(self.maze_matrix)
            path = []
            
            for v_coords in graph_path:
                v_x1 = int(v_coords[0])
                v_x2 = int(v_coords[1])
                
                if (solved_matrix[v_x1][v_x2] != 'S' and solved_matrix[v_x1][v_x2] != 'F'):
                    solved_matrix[v_x1][v_x2] = '*'
                
                path.append((v_x1, v_x2))
            return (path, solved_matrix)
        
                
maze_matrix = [['S','#','.'],
               ['.','X','.'],
               ['.','#','F']]        
        
maze = Maze(maze_matrix)

# print(maze)
# print(maze.astar(maze.start, maze.finish))
(path, solved_matrix) = maze.solve()

print(path)

print()

for row in solved_matrix:
    print(row)


Found the path!
[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]

['S', '*', '*']
['.', 'X', '*']
['.', '#', 'F']
